This notebook shows how all the pieces of CatSim fit together without much
attempt at explanation. The general point of the CatSim software is to produce a catalog of objects
visible through a given telescope at a given time from a given position.

The objects are read in from a database which is handled by the <b>CatalogDBObject</b>
class.   The parameters of the telescope pointing (direction, date, telescope location)
are handled by the <b>ObservationMetaData</b> class.  This information is combined and output as a catalog by the <b>InstanceCatalog</b> class.

Below we walk through a cartoon example, instantiating each of these classes one
at a time.

<b>CatalogDBObject</b>

<b>CatalogDBObject</b> is a class that connects our python code to database tables.
It is defined in

sims_catalogs_generation/python/lsst/sims/catalogs/generation/db/dbConnection.py

Different daughter classes of this method have been written to interface with specific tables
in the fatboy databases.  <b>CatalogDBObject</b> contains a class method `from_objid()`
which allows you to instantiate these daughter classes by referring to their
`objid` member variable.  The daughter classes insantiated below are defined in

sims_catUtils/python/lsst/sims/catUtils/baseCatalogModels/GalaxyModels.py

sims_catUtils/python/lsst/sims/catUtils/baseCatalogModels/StarModels.py

There is also a file in that directory that defines an interface to a table
of solar system objects and one that defines an interface to the Opsim 3.61 run.


In [ ]:
from lsst.sims.catalogs.generation.db import CatalogDBObject
from lsst.sims.catUtils.baseCatalogModels import *

myGalaxyDB = CatalogDBObject.from_objid('galaxyTiled')
myStarDB = CatalogDBObject.from_objid('allstars')

<b> ObservationMetaData </b>

The <b>ObservationMetaData</b> class defines a particular pointing of the telescope.
The <b>InstanceCatalog</b> classes below will end up querying our <b>CatalogDBObject</b>
and using an <b>ObservationMetaData</b> object to constrain that query so that
the written catalog only contains objects that are visible from the telescope
at a given time, in a given direction, and within a given field of view size (see the caveat at the end of this cell).

The <b>ObservationMetaData</b> class is defined in

sims_utils/python/lsst/sims/utils/ObservationMetaData.py

Generally, an <b>ObservationMetaData</b> is defined by a `pointingRA`, `pointingDec` (in degrees), a `boundType`
(either 'circle' or 'box') defining the shape of the field of view, a `boundLength`
(either a float or a numpy array) defining the size of the field of view in degrees,
and an `mjd` defining the date of the observation.

You can also specify a `site` which is an instantiation of the <b>Site</b> class defined in

sims_utils/python/lsst/sims/utils/Site.py

which characterizes the telescope's location.  This defaults to the LSST site.

There are other optional arguments, mostly related to interfacing the catalog with
PhoSim.  See the class's docstring for more detailed information.

<b>Note:</b> Because CatSim assumes that the RA, Dec of objects stored in the database are defined in the International Celestial Reference System (ICRS), which neglects precession, nutation, aberration, and atmospheric refraction, the `ObservationMetaData` will query a field of view slightly larger than the one requested, in order to ensure that the query returns all objects which are actually visible in the field of view after those coordinate effects have been applied.

In [ ]:
from lsst.sims.utils import ObservationMetaData

obs_metadata = ObservationMetaData(pointingRA = 220.0,
                                   pointingDec = 19.0,
                                   boundType = 'circle',
                                   boundLength = 0.2,
                                   mjd = 52000.0)

<b>InstanceCatalog</b>

The <b>InstanceCatalog</b> class is defined in

sims_catalogs_measures/python/lsst/sims/catalogs/measures/instance/InstanceCatalog.py

The <b>InstanceCatalog</b> (or daughter classes thereof) define what data should be output
to the catalog (i.e. do you just want ra and dec, or do you also want magnitudes integrated
over your telescope's bandpasses, the name of the camera chip that actually sees the object,
etc.)  Tutorials 01 and 02 will show how the <b>InstanceCatalog</b> actually gets and processes
this data.  Below, we will just demonstrate the user interface.

The daughter classes of <b>InstanceCatalog</b> used below are defined in

sims_catUtils/python/lsst/sims/catUtils/exampleCatalogDefinitions/refCatalogExamples.py

This portion of the script will result in two output files

star_example.txt

galaxy_example.txt

being written to the current working directory (the one contains only stars; the other
contains only galaxies)

In [ ]:
from lsst.sims.catUtils.exampleCatalogDefinitions import RefCatalogGalaxyBase, \
                                                  RefCatalogStarBase

myStarCat = RefCatalogStarBase(myStarDB, obs_metadata=obs_metadata)
myStarCat.write_catalog('star_example.txt')

myGalaxyCat = RefCatalogGalaxyBase(myGalaxyDB, obs_metadata=obs_metadata)
myGalaxyCat.write_catalog('galaxy_example.txt')

<b>Alternate ObservationMetaData</b>

Above we used an <b>ObservationMetaData</b> object with a circular bound on the field of view.
Below we try a square field of view, just so you can see that the results actually
come out as advertised.

This portion of the script will write

star_example_square.txt

to the current working directory

In [ ]:
squareObsMetadata = ObservationMetaData(pointingRA = 220.0,
                                       pointingDec = 19.0,
                                       boundType = 'box',
                                       boundLength = 0.3,
                                       mjd = 52000.0)

myStarCat = RefCatalogStarBase(myStarDB, obs_metadata=squareObsMetadata)
myStarCat.write_catalog('star_example_square.txt')